In [115]:
import numpy as np
import pandas as pd
#from scipy import stats
#import seaborn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [127]:
data=pd.read_csv('orange_small_churn_data.txt')
data.shape

(40000, 230)

In [128]:
label=pd.read_csv('orange_small_churn_labels.txt',header=None)
label.shape

(40000, 1)

In [129]:
label.columns=['label']
X=data.iloc[:28000,:]
y=label.iloc[:28000,:]
X_holdout=data.iloc[-12000:,:]
y_holdout=label.iloc[-12000:,:]
print X.shape,X_holdout.shape
print y.shape,y_holdout.shape

(28000, 230) (12000, 230)
(28000, 1) (12000, 1)


In [130]:
#Отбросим колонки где данных вообще нет
zero=list()
s=X.describe()
for var in s.columns:
    if s[var]['count']==0:zero.append(var)
print zero
print 'Колонки ,которые мы отбросили', len(zero)
real_columns=X.columns.drop(zero)
X=X[real_columns]

['Var8', 'Var15', 'Var20', 'Var31', 'Var32', 'Var39', 'Var42', 'Var48', 'Var52', 'Var55', 'Var79', 'Var141', 'Var167', 'Var169', 'Var175', 'Var185', 'Var209', 'Var230']
Колонки ,которые мы отбросили 18


In [131]:
X.shape

(28000, 212)

In [132]:
#Из 230 колонок выбросли уже 18 где нет данных(из них 16 вещ. и 2 кат.).190(последняя вещ.)-16=174
num_data=X.iloc[:,:174]
cat_data=X.iloc[:,174:]
print num_data.shape
print y.shape


(28000, 174)
(28000, 1)


In [133]:
#Разница в значениях вещ.признаков достигает нескольких порядков!! -применим масштабирование
#Заполним средним по столбцу значения NaN иначе scale не сработает
num_data_mean=num_data.fillna(num_data.describe().loc['mean'])
#num_scaled=pd.DataFrame(scale(num_data_mean),columns=num_data.columns)

In [134]:
num_data_mean

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
0,10.129199,0.007194,499.301153,0.144165,261114.121795,3052.000000,6.775233,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
1,10.129199,0.007194,499.301153,0.144165,261114.121795,1813.000000,7.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,276.000000,22601.892357
2,10.129199,0.007194,499.301153,0.144165,261114.121795,1953.000000,7.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
3,10.129199,0.007194,499.301153,0.144165,261114.121795,1533.000000,7.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
4,10.129199,0.007194,499.301153,0.144165,261114.121795,686.000000,7.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
5,0.000000,0.007194,499.301153,0.144165,261114.121795,1302.354079,6.775233,30.000000,399938.134615,8.576369,...,5.017544e+06,0.592661,1.382283e+06,74948.32518,7.435971,0.000000,22.000000,164.73341,174.000000,22601.892357
6,10.129199,0.007194,499.301153,0.144165,261114.121795,889.000000,0.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
7,10.129199,0.007194,499.301153,0.144165,261114.121795,161.000000,0.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,252.000000,22601.892357
8,10.129199,0.007194,499.301153,0.144165,261114.121795,420.000000,0.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357
9,10.129199,0.007194,499.301153,0.144165,261114.121795,1155.000000,0.000000,46.496124,399938.134615,8.576369,...,3.810382e+06,0.000000,1.382283e+06,74948.32518,7.435971,3.627907,15.906977,164.73341,270.156266,22601.892357


In [38]:
#Поделим выборку на классы, чтобы посчитать корреляции для вещ. и категориального признаков
num_scaled_data=num_scaled.join(y)
num_scaled_data1=num_scaled_data[num_scaled_data.label==1.0]
num_scaled_data0=num_scaled_data[num_scaled_data.label==-1.0]
print num_scaled_data1.shape
print num_scaled_data0.shape
#Классы несбалансированы!

(2101, 175)
(25899, 175)


In [39]:
#Считаем корреляции для вещ.
corr=list()
for var in num_scaled_data.columns:
    c=num_scaled_data1[var].mean()-num_scaled_data0[var].mean()
    corr.append(c)
#Записываем в последнюю строку
num_scaled_data.loc[len(num_scaled_data)]=corr
#Выделяем эту строку с названиями переменных
res=num_scaled_data.drop('label',1).iloc[28000:,:].T
print res

           28000
Var1    0.007746
Var2   -0.003595
Var3   -0.002740
Var4    0.008830
Var5   -0.006784
Var6   -0.078762
Var7   -0.260574
Var9    0.000525
Var10   0.024959
Var11  -0.010731
Var12   0.018655
Var13  -0.165978
Var14  -0.019490
Var16  -0.009107
Var17  -0.006464
Var18  -0.003081
Var19   0.031538
Var21  -0.029223
Var22  -0.029546
Var23  -0.009849
Var24  -0.033564
Var25  -0.032096
Var26  -0.001621
Var27  -0.014405
Var28   0.050863
Var29  -0.006015
Var30  -0.002595
Var33  -0.020850
Var34  -0.013054
Var35   0.038769
...          ...
Var157 -0.008624
Var158 -0.003819
Var159  0.002397
Var160 -0.050888
Var161  0.019954
Var162  0.007266
Var163  0.008028
Var164 -0.008257
Var165 -0.003755
Var166 -0.013313
Var168 -0.082873
Var170  0.003971
Var171 -0.005765
Var172 -0.013412
Var173  0.043163
Var174  0.001775
Var176 -0.015053
Var177  0.031914
Var178 -0.017099
Var179  0.005075
Var180  0.010253
Var181 -0.002389
Var182 -0.007182
Var183  0.009039
Var184  0.005241
Var186  0.004563
Var187  0.0028

In [40]:
#Выбираем 20 наиболее коррелирущих числовых переменных
top20=(np.absolute(res)).nlargest(20,28000).T
print top20

         Var189     Var73      Var7    Var113    Var144     Var13     Var65  \
28000  0.316838  0.274598  0.260574  0.210127  0.193529  0.165978  0.162428   

        Var126     Var81     Var74     Var72    Var140    Var168      Var6  \
28000  0.15285  0.149044  0.127481  0.117881  0.101394  0.082873  0.078762   

         Var143    Var160     Var28    Var131    Var125    Var139  
28000  0.064594  0.050888  0.050863  0.050493  0.050304  0.046215  


In [41]:
#Удалим 65 - сильно коррелирует с 7 и 72
corr = num_data[top20.columns].corr()
print corr[(corr > 0.64) * (corr < 1.0)].dropna(axis=1,how="all")

            Var7     Var65     Var72
Var189       NaN       NaN       NaN
Var73        NaN       NaN       NaN
Var7         NaN  0.641425       NaN
Var113       NaN       NaN       NaN
Var144       NaN       NaN       NaN
Var13        NaN       NaN       NaN
Var65   0.641425       NaN  0.668722
Var126       NaN       NaN       NaN
Var81        NaN       NaN       NaN
Var74        NaN       NaN       NaN
Var72        NaN  0.668722       NaN
Var140       NaN       NaN       NaN
Var168       NaN       NaN       NaN
Var6         NaN       NaN       NaN
Var143       NaN       NaN       NaN
Var160       NaN       NaN       NaN
Var28        NaN       NaN       NaN
Var131       NaN       NaN       NaN
Var125       NaN       NaN       NaN
Var139       NaN       NaN       NaN


In [42]:
top19=top20.columns.drop('Var65')#удалили 65

In [43]:
num=num_scaled_data[top19].iloc[:28000,:]# выборка с вещественными признаками
num

,Var189,Var73,Var7,Var113,Var144,Var13,Var126,Var81,Var74,Var72,Var140,Var168,Var6,Var143,Var160,Var28,Var131,Var125,Var139
0,0.000000,-0.617318,0.000000,-1.389937,-0.244123,0.000000,2.366933e-01,-8.774688e-01,-4.200410e-17,0.000000,5.331969e-17,4.864750e-15,0.747646,-9.755851e-02,-0.121361,-2.677233e-01,0.000000,0.000000,1.323802e-15
1,0.103905,1.166375,0.037845,0.745863,0.567744,-0.235159,2.120381e+00,-3.518477e-01,-2.977063e-01,-0.693173,-2.764670e-01,4.864750e-15,0.218205,-9.755851e-02,-0.275596,-6.411942e-01,0.000000,-0.039285,1.323802e-15
2,0.000000,1.887443,0.037845,0.033987,1.379611,-0.307607,1.911082e+00,-1.757086e-01,4.264588e-01,-0.693173,-2.307390e-01,4.864750e-15,0.278029,-9.755851e-02,-0.231529,9.804561e-01,0.000000,-0.238030,1.323802e-15
3,0.000000,-0.693220,0.037845,0.694093,-1.055989,-0.478707,5.809204e-18,-2.865227e-01,-2.977063e-01,0.000000,-3.233675e-01,4.864750e-15,0.098558,-9.755851e-02,0.759978,-4.346198e-02,0.000000,-0.311321,1.323802e-15
4,0.000000,-0.655269,0.037845,0.384629,-0.244123,-0.480249,-1.437696e+00,6.764099e-01,-2.977063e-01,-0.693173,-2.717770e-01,4.864750e-15,-0.263376,-9.755851e-02,-0.165428,6.034112e-01,0.000000,-0.313849,1.323802e-15
5,-1.709709,-1.186581,0.000000,-4.371550,0.000000,0.000000,-1.019099e+00,3.034710e-15,-4.200410e-17,0.000000,5.331969e-17,1.015289e+01,0.000000,-2.159396e-17,0.000000,-1.364926e-13,-1.399391,0.000000,1.323802e-15
6,0.000000,-0.769121,-1.140776,0.362670,-0.244123,-0.480249,2.366933e-01,-8.965452e-02,-2.977063e-01,1.060367,-3.245400e-01,4.864750e-15,-0.176631,-9.755851e-02,-0.231529,1.076951e+00,0.000000,-0.319106,1.323802e-15
7,-0.322828,-0.920925,-1.140776,0.661401,-1.055989,-0.480249,2.366933e-01,1.339737e+00,-2.977063e-01,-0.693173,-3.245400e-01,4.864750e-15,-0.487715,-9.755851e-02,-0.165428,7.034800e-01,0.000000,-0.319106,1.323802e-15
8,0.000000,-0.845023,-1.140776,0.146907,-1.055989,-0.480249,2.366933e-01,-1.002484e+00,-2.977063e-01,-0.693173,-3.245400e-01,4.864750e-15,-0.377041,-9.755851e-02,0.297275,-2.677233e-01,0.000000,-0.319106,1.323802e-15
9,0.000000,-0.731170,-1.140776,0.136908,-1.055989,-0.480249,2.366933e-01,-9.829568e-01,-2.977063e-01,-0.693173,-3.245400e-01,4.864750e-15,-0.062966,-9.755851e-02,0.297275,-6.411942e-01,0.000000,-0.319106,1.323802e-15


In [135]:
cat_data=cat_data.fillna('NA')
#cat_data=cat_data.join(y)

In [45]:
#Отбираем категориальные по Крамеру с достижением значимости <0.05
k=0
new=list()
from scipy import stats
from math import sqrt
for var in cat_data.columns.drop(label):
    subtable=pd.crosstab(cat_data[var], cat_data.label)
    ch2,p,dof,e=stats.chi2_contingency(subtable,correction=False)
    cramer=sqrt(ch2/(sum(subtable.sum())-1))
    if p<0.05 and cramer>0.07 and len(cat_data[var].value_counts())<100 : 
        k+=1
        print 'Var ,p,cramer',var,p,cramer
        print len(cat_data[var].value_counts())
        new.append(var)
print k
print new
cat=cat_data[new]
print cat

Var ,p,cramer Var193 3.55647398956e-13 0.0739638072254
48
Var ,p,cramer Var205 6.86475793867e-32 0.0727219318482
4
Var ,p,cramer Var206 1.79179794029e-31 0.0848445723764
22
Var ,p,cramer Var212 1.69160282443e-21 0.0951564396571
75
Var ,p,cramer Var218 1.42205763577e-67 0.104855885368
3
Var ,p,cramer Var228 2.03000048467e-22 0.0785866281257
30
6
['Var193', 'Var205', 'Var206', 'Var212', 'Var218', 'Var228']
                       Var193   Var205   Var206         Var212 Var218  \
0                     AERks4l     09_Q     IYzP     JBfYVit4g8   cJvF   
1                     2Knk1KF     VpdQ     haYg  XfqtO3UdzaXh_   cJvF   
2            LrdZy8QqgUfkVShG     VpdQ     hAFG  4kVnq_T26xq1p   UYBR   
3                        RO12     VpdQ     IYzP        NhsEn4L   cJvF   
4                        RO12  sJzTlal     zm5i        NhsEn4L   cJvF   
5                        RO12     09_Q       NA           CrNX   cJvF   
6                        RO12  sJzTlal     zm5i        NhsEn4L   UYBR   
7       

In [136]:
from sklearn.feature_extraction import DictVectorizer
encoder = DictVectorizer(sparse = False)
cat_code = encoder.fit_transform(cat.T.to_dict().values())
print (cat_code)

[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [137]:
cat_code.shape

(28000L, 182L)

In [138]:
X_select=np.hstack( (num_data_mean,cat_code))
X_select.shape

(28000L, 356L)

In [139]:
(X_train,  X_test,  y_train, y_test) = train_test_split(X_select, np.array(y), test_size=0.3, random_state=0)
print (X_train.shape)
print (X_test.shape)

(19600L, 356L)
(8400L, 356L)


In [118]:
X_train

array([[ 0.53063717,  0.14170059,  2.39508694, ...,  0.        ,
         0.        ,  0.        ],
       [-0.42951101, -0.69321957,  0.03784498, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.74400343,  0.407357  ,  2.39508694, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [-0.21614475, -0.27575949, -1.140776  , ...,  0.        ,
         0.        ,  0.        ],
       [-0.21614475,  0.93866983,  2.39508694, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  3.02597023,  0.03784498, ...,  0.        ,
         0.        ,  0.        ]])

In [140]:
y_train=y_train.reshape(19600,)
y_test=y_test.reshape(8400,)
y_test

array([-1, -1, -1, ..., -1, -1, -1], dtype=int64)

In [141]:
def metr(a,b):
    print metrics.classification_report(a,b)
    print 'accuracy', metrics.accuracy_score(a,b)
    print 'roc-auc',metrics.roc_auc_score(a,b)
    print 'PR AUC',metrics.average_precision_score(a,b)

In [146]:
clf=RidgeClassifier(class_weight='balanced',random_state=1)
cv=StratifiedKFold(n_splits=3)
clf.fit(X_train,y_train)
RC=cross_val_score(clf,X_train,y_train)
print RC
pred=clf.predict(X_test)
metr(y_test, pred)
clf.coef_

[ 0.60896847  0.60615338  0.61656207]
             precision    recall  f1-score   support

         -1       0.95      0.60      0.73      7757
          1       0.11      0.60      0.19       643

avg / total       0.88      0.60      0.69      8400

accuracy 0.599880952381
roc-auc 0.602217612708
PR AUC 0.373242394986


array([[  4.48118776e-03,   6.49585660e-02,  -2.27442759e-06,
          5.59236737e-02,   1.15376072e-08,  -1.67778832e-06,
         -4.95647822e-03,   2.23426681e-02,   9.02266198e-08,
         -2.54028355e-02,   1.71635657e-02,  -1.30434488e-06,
         -2.76342206e-02,  -1.16052008e-03,  -1.76497713e-02,
          1.06294366e-02,   8.09382387e-02,   1.15234944e-03,
         -1.16500474e-03,   5.22594621e-03,  -1.13611172e-03,
          2.33479395e-04,   2.01792613e-01,  -9.22080954e-02,
          1.33021858e-06,   1.19442798e-02,  -3.86084737e-02,
         -5.61184159e-07,  -8.79875555e-03,  -2.24674979e-02,
          2.16778892e-07,  -4.02779359e-08,   5.82815635e-09,
         -4.63362567e-04,   2.52600770e-03,  -2.22745809e-02,
         -2.34701922e-02,  -2.54317338e-05,  -6.46911297e-03,
          4.13690929e-03,  -7.51270502e-02,   1.70396950e-04,
         -4.03005078e-07,   2.17237114e-07,   2.98831699e-02,
          8.33946403e-08,  -4.06390127e-03,  -2.33459613e-08,
        

In [143]:
clf=RandomForestClassifier(class_weight='balanced',random_state=1)
cv=StratifiedKFold(n_splits=3)
clf.fit(X_train,y_train)
RC=cross_val_score(clf,X_train,y_train)
print RC
pred=clf.predict(X_test)
metr(y_test, pred)

[ 0.92470156  0.92530231  0.92514924]
             precision    recall  f1-score   support

         -1       0.92      1.00      0.96      7757
          1       0.11      0.00      0.00       643

avg / total       0.86      0.92      0.89      8400

accuracy 0.922619047619
roc-auc 0.500261941705
PR AUC 0.0945474462465


In [144]:
clf=GradientBoostingClassifier(random_state=1)
cv=StratifiedKFold(n_splits=3)
clf.fit(X_train,y_train)
RC=cross_val_score(clf,X_train,y_train)
print RC
pred=clf.predict(X_test)
metr(y_test, pred)

[ 0.92500765  0.92407776  0.9248431 ]
             precision    recall  f1-score   support

         -1       0.92      1.00      0.96      7757
          1       0.60      0.01      0.03       643

avg / total       0.90      0.92      0.89      8400

accuracy 0.92380952381
roc-auc 0.506611697336
PR AUC 0.344736540028


In [145]:
clf=SGDClassifier(random_state=1,class_weight='balanced')
cv=StratifiedKFold(n_splits=3)
clf.fit(X_train,y_train)
RC=cross_val_score(clf,X_train,y_train)
print RC
pred=clf.predict(X_test)
metr(y_test, pred)

[ 0.32736455  0.70978111  0.07684066]
             precision    recall  f1-score   support

         -1       0.92      0.64      0.75      7757
          1       0.07      0.31      0.11       643

avg / total       0.85      0.62      0.71      8400

accuracy 0.615119047619
roc-auc 0.475683429265
PR AUC 0.215334859007


In [ ]:
np.random.seed(0)
indices_to_add = np.random.randint(1884,size=(np.sum(y_train==0)-np.sum(y_train==1)))
#print X_train_scaled_cat.shape
X_train_to_add =X_train_scaled_cat[y_train.as_matrix() == 1,:][indices_to_add,:]
#print X_train_to_add.shape
#print y_train.shape
X_add=np.vstack( (X_train_scaled_cat,X_train_to_add) )
#print X_add.shape
ones= np.ones((432))
y_add=np.hstack((y_train, ones))
#print y_add.shape